# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — ежемесячный доход

purpose — цель получения кредита


## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
data=pd.read_csv('/datasets/data.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

-В столбцах "days_employed" и "total_income" явно имеются пропуски;

**По первым 10 строкам датасета можно выделить следующие проблемы:**

-В столбце о стаже работы имеются отрицательные значения;

-Столбец "образование" заполнен разным регистром.


## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data['children'].value_counts() #уникальные значения для столбца дети.

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Ошибка в данных: -1 и 20 детей.

In [3]:
data['days_employed'].value_counts()  #уникальные значения для столбца рабочий стаж.

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64

 Отрицательные значения и float64

In [4]:
data['dob_years'].value_counts()  #уникальные значения для столбца возраст.

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

101 нулевых значений.

In [5]:
data['education'].value_counts()  #уникальные значения для столбца образование. 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Разный регистр= явные дубликаты

In [6]:
data['education_id'].value_counts()  #уникальные значения для столбца идентификатор образования. Он совпадает с предыдущим.

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [7]:
data['family_status'].value_counts()  #уникальные значения для столбца семейное положение. Проблем не вижу.

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [8]:
data['family_status_id'].value_counts()  #уникальные значения для столбца идентификатор семейного положения. совпадает с предыдущим столбцом.

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [9]:
data['gender'].value_counts()  #уникальные значения для столбца пол.

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Одна строка не заполнена.

In [10]:
data['income_type'].value_counts()#уникальные значения для столбца тип занятости. Всё хорошо.

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [11]:
data['debt'].value_counts() #уникальные значения для столбца задолжность. Всё в порядке.

0    19784
1     1741
Name: debt, dtype: int64

In [12]:
data['total_income'].value_counts() #уникальные значения для столбца ежемесячный заработок. 

112874.418757    1
133912.272223    1
182036.676828    1
122421.963500    1
198271.837248    1
                ..
133299.194693    1
115080.782380    1
84896.781597     1
153838.839212    1
150014.128510    1
Name: total_income, Length: 19351, dtype: int64

Слишком много знаков после запятой.

In [13]:
data['purpose'].value_counts()  #уникальные значения для столбца цель кредита. 

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Много скрытых повторов.

In [14]:
data['children']=data['children'].replace(20, 2) #слишком много детей
data['children']=data['children'].replace(-1, 1) #слишком мало детей

#Заполняем медианами пропуски столбца с доходами в зависимости от типа занятости.
data['median_income'] = data.groupby('income_type')['total_income'].transform('median')
data['total_income'] = data['total_income'].fillna(data['median_income'])

#Заполняем медианами пропуски столбца трудовой стаж в зависимости от типа занятости.
data['median_days_employed']=data.groupby('income_type')['days_employed'].transform('median')
data['days_employed'] = data['total_income'].fillna(data['median_days_employed'])

#Удаляем строку с непонятным гендером
data = data[data['gender'] != 'XNA']

#Приводим трудовой стаж к целым положительным числам
data['days_employed']=data['days_employed'].astype('int')
data['days_employed'] = data['days_employed'].abs()
#Заменяем нули в стобце возраст на медиану в зависимости от трудового стажа. 
income_type_gr = data.groupby('income_type')['dob_years'].median()

data.loc[(data['income_type'] == 'безработный ') & (data['dob_years'] == 0), 'dob_years'] = 38
data.loc[(data['income_type'] == 'в декрете') & (data['dob_years'] == 0), 'dob_years'] = 39
data.loc[(data['income_type'] == 'госслужащий') & (data['dob_years'] == 0), 'dob_years'] = 40
data.loc[(data['income_type'] == 'компаньон') & (data['dob_years'] == 0), 'dob_years'] = 39
data.loc[(data['income_type'] == 'пенсионер') & (data['dob_years'] == 0), 'dob_years'] = 60
data.loc[(data['income_type'] == 'предприниматель') & (data['dob_years'] == 0), 'dob_years'] = 42.5
data.loc[(data['income_type'] == 'сотрудник') & (data['dob_years'] == 0), 'dob_years'] = 39
data.loc[(data['income_type'] == 'студент') & (data['dob_years'] == 0), 'dob_years'] = 22

data['dob_years']=data['dob_years'].astype('int')

data['total_income']=data['total_income'].astype('int')


**Вывод**

У кого-то 20 и -1 ребенок (человеческий фактор)

В столбце гендер одна строка с пропуском. Откинем ее.

В столбце возраст 101 нулевых значений. Заполним медианой значениями исходя из типа занятости. (технические пропуски)

В столбцах "days_employed" и "total_income" 2174 пропуска. Заменяем медианой исходя из типа занятости. (технические пропуски)

Столбец 'education' приведем к нижнему регистру.

В столбце цель кредита, много скрытых повторов.

### Замена типа данных

**Вывод**

**Заполнены все пропуски в данных**

**Данные приведены к подходящему для обработки виду**

### Обработка дубликатов

In [15]:
data.duplicated().sum()

54

In [16]:
data=data.drop_duplicates().reset_index(drop=True)

**Вывод**

Удалили дубликаты и обновили индексацию. 

### Лемматизация

In [17]:
#пишем функцию поиска лемм
def lemma(lem):
    purpose= ' ' .join(m.lemmatize(lem))
    return purpose

#применяем к столбцу цель
data['lem_purpose']=data['purpose'].apply(lemma) 

#смотрим все варианты
data['lem_purpose'].unique()     

array(['покупка   жилье \n', 'приобретение   автомобиль \n',
       'дополнительный   образование \n', 'сыграть   свадьба \n',
       'операция   с   жилье \n', 'образование \n',
       'на   проведение   свадьба \n', 'покупка   жилье   для   семья \n',
       'покупка   недвижимость \n',
       'покупка   коммерческий   недвижимость \n',
       'покупка   жилой   недвижимость \n',
       'строительство   собственный   недвижимость \n', 'недвижимость \n',
       'строительство   недвижимость \n',
       'на   покупка   подержать   автомобиль \n',
       'на   покупка   свой   автомобиль \n',
       'операция   с   коммерческий   недвижимость \n',
       'строительство   жилой   недвижимость \n', 'жилье \n',
       'операция   со   свой   недвижимость \n', 'автомобиль \n',
       'заниматься   образование \n',
       'сделка   с   подержанный   автомобиль \n',
       'получение   образование \n', 'свадьба \n',
       'получение   дополнительный   образование \n',
       'покупка   свой 

**Вывод**

После леммизации можно выделить 4 основные группы цели кредита:

-Недвижимость;

-Автомобиль;

-Образование; 

-Свадьба.

### Категоризация данных

In [18]:
#пишем функцию преобразования целей
def purpose_word(word):
    if 'жилье' in  word:
        return 'недвижимость'
    if 'недвижимость' in  word:
        return 'недвижимость'
    if 'автомобиль' in  word:
        return 'автомобиль'
    if 'свадьба' in  word:
        return 'свадьба'
    if 'образование' in  word:
        return 'образование'

#заполняем столбец цели одним словом функцией
data['purpose']=data['lem_purpose'].apply(purpose_word)

display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,median_days_employed,lem_purpose
0,1,253875,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,142594.396847,-1574.202821,покупка жилье \n
1,1,112080,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,142594.396847,-1574.202821,приобретение автомобиль \n
2,0,145885,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,142594.396847,-1574.202821,покупка жилье \n
3,3,267628,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,142594.396847,-1574.202821,дополнительный образование \n
4,0,158616,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,118514.486412,365213.306266,сыграть свадьба \n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21465,1,224791,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,недвижимость,172357.950966,-1547.382223,операция с жилье \n
21466,0,155999,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль,118514.486412,365213.306266,сделка с автомобиль \n
21467,1,89672,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,142594.396847,-1574.202821,недвижимость \n
21468,3,244093,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль,142594.396847,-1574.202821,на покупка свой автомобиль \n


**Вывод**

Данные в датафрейме отчищены от пропусков, дубликатов и сгруппированы.

Можно приступать к анализу.



## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
data_piv=data.pivot_table(index=['children'], columns='debt', values = 'gender', aggfunc='count')
data_piv['share']=(data_piv[1]/(data_piv[1]+data_piv[0]))*100
data_piv



debt,0,1,share
children,,,
0,13043.0,1063.0,7.535800
1,4411.0,445.0,9.163921
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


**Вывод**

Вероятность задержки платежей по кредиту увеличивается при наличии детей.

*В свою очередь, количество детей не влияет на эту величину.*

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [20]:
#тоже самое для семейного положения
data_piv1 = data.pivot_table(index=['family_status'], columns='debt', values = 'gender', aggfunc='count')
data_piv1['share']=(data_piv1[1]/(data_piv1[1]+data_piv1[0]))*100
data_piv1

debt,0,1,share
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3774,388,9.322441
женат / замужем,11413,931,7.542126


**Вывод**

По полученным данным можно сделать выдвод, что не женатые и в гражданском браке чаще задерживают выплаты по кредитам.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
pd.qcut(data['total_income'], 4)

#напишем функцию для группировки по уровню дохода
def income_group(inc):
    
    income=inc['total_income']
    
    if income<=107652.25:
        
        return 'до 107652.25'
    
    if (income>107652.25) & (income<=142594):
        
        return 'от 107652.25 до 142594'
    
    if income>142594 and income<=195751.75:
        
        return '142594. до 195751.75'
    
    if income>195751.75:
        
        return 'от 195751.75'
    
#Применяем и записываем в новый столбец
data['income_group']=data.apply(income_group,axis=1)

#делаем сводную по доходу
data_piv2 = data.pivot_table(index=['income_group'], columns='debt', values = 'gender', aggfunc='count')
data_piv2['share']=(data_piv2[1]/(data_piv2[1]+data_piv2[0]))*100
data_piv2

debt,0,1,share
income_group,,,
142594. до 195751.75,4796,448,8.543097
до 107652.25,4941,427,7.954545
от 107652.25 до 142594,5007,483,8.797814
от 195751.75,4985,383,7.134873


**Вывод**

Чаще всего просрочки по кредитам можно ждать от людей с доходом выше среднего.

- Как разные цели кредита влияют на его возврат в срок?

In [22]:
data_piv3 = data.pivot_table(index=['purpose'], columns='debt', values = 'gender', aggfunc='count')
data_piv3['share']=(data_piv3[1]/(data_piv3[1]+data_piv3[0]))*100
data_piv3

debt,0,1,share
purpose,,,
автомобиль,3905,403,9.354689
недвижимость,10031,782,7.232036
образование,3644,370,9.217738
свадьба,2149,186,7.965739


**Вывод**

Вероятность задержки возрастает при покупке автомобиля или кредита на образование

## Шаг 4. Общий вывод

# Проведенное исследование позволило выявить следующие зависимости:

1. При наличии у заемщика детей возрастает вероятность задолжностей (9.1% невозврата);
2. Холостяки, а так же люди, состоящие в гражданском браке чаще просрачивают платежи по кредитам(9.5% невозврата);
3. Люди с ежемесячным достатком от 107652.25 руб до 142594 руб. чаще просрачивают платежи по кредитам(8,6% невозврата);
4. Кредиты на покупку автомобиля(9.3% невозврата) и плату за обучение (9.2% невозврата) чаще остаются не выпраченными.

Исходя из данных выводов, можно сказать, что наиболее надежным заемщиком является:

- Женатый
- Без детей.
- С доходом до 107652.25 руб., либо с доходом от 195751.75 руб. 
- С целью кредита: Свадьба или Недвижимость.
